In [1]:
pip install langchain langchain-openai langchain-community langchain-text-splitters sentence-transformers pypdfium2 chromadb langchain-chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### 임베딩 모델

In [210]:
import shutil
import os
from pathlib import Path

# 전체 Chroma DB 삭제 (기본 디렉토리와 시스템 캐시 포함)
shutil.rmtree("./chroma_db", ignore_errors=True)
shutil.rmtree(Path.home() / ".chromadb", ignore_errors=True)

import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb

os.environ["OPENAI_API_KEY"] = "키"

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
client = chromadb.PersistentClient(path="./chroma_db")

def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)

        #시나리오
        for item in data:
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={
                        "카테고리": item.get("카테고리", "")
                    }
                ))

            # 수강이력
            if "lecture_name" in item and "student_id" in item:
                content = (
                    f"학번: {item['student_id']}\n"
                    f"강의명: {item['lecture_name']}\n"
                    f"학정번호: {item['lecture_id']}\n"
                    f"개설 학과: {item['department_offered']}\n"
                    f"이수 구분: {item['lecture_course_type']}\n"
                    f"학점: {item['lecture_credit']}학점\n"
                    f"성적: {item['lecutre_grade']}"
                )
                if "retake_or_delete_status" in item:
                    content += f"\n재수강/삭제 여부: {item['retake_or_delete_status']}"
                if "retake_status" in item:
                    content += f"\n재수강 여부: {item['retake_status']}"

                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_name": item["lecture_name"],
                        "course_type": item["lecture_course_type"]
                    }
                ))

            # 강의 탐색
            if "lecture_id" in item and "lecture_name" in item and "student_id" not in item:
                content = (
                    f"학정번호: {item.get('lecture_id', '')}\n"
                    f"강의명: {item.get('lecture_name', '')}\n"
                    f"강의평점: {item.get('lecture_ratings', '')}\n"
                    f"과제: {item.get('lecture_homework', '')}\n"
                    f"팀플: {item.get('lecture_team', '')}\n"
                    f"성적평가정도: {item.get('lecutre_grade', '')}\n"
                    f"출결 방식: {item.get('lecutre_attendance', '')}\n"
                    f"시험 횟수: {item.get('lecutre_test', '')}\n"
                    f"시험 방식: {item.get('lecture_testinform', '')}\n"
                    f"전공 학점: {item.get('credits_major', '없음')}\n"
                    f"교양 학점: {item.get('credits_general', '없음')}\n"
                    f"총 학점: {item.get('credits_total', '없음')}\n"
                    f"교수명: {item.get('lecture_professorname', '')}\n"
                    f"수업 시간: {item.get('lecture_time', '')}\n"
                    f"강의 유형: {item.get('lecture_course_type', '')}\n"
                    f"학점: {item.get('lecture_hours', '')}시간\n"
                    f"학기: {item.get('lecture_semester', '')}학기\n"
                    f"강의 설명: {item.get('lecture_inform', '')}"
                    f"영역: {item.get('lecture_domain', '')}\n"
                )
                docs.append(Document(
                    page_content=content,
                    metadata={
                        "lecture_id": item.get("lecture_id", ""),
                        "lecture_name": item.get("lecture_name", ""),
                        "lecture_ratings": item.get("lecture_ratings", ""),
                        "lecture_team": item.get("lecture_team", ""),
                        "lecutre_grade": item.get("lecutre_grade", ""),
                        "professor": item.get("lecture_professorname", "")
                    }
                ))
    return docs

user_datasets = {
    "kim": ["data/kw_chatbot_data - 김브티_수강이력.json"],
    "hong": ["data/kw_chatbot_data - 홍데사_수강이력.json"]
}

task_datasets = {
    "lecture_search": [
        "data/kw_chatbot_data - Student.json",
        "data/kw_chatbot_data - 강의 평점.json",
        "data/kw_chatbot_data - 강의계획서.json",
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 커리큘럼(DS).json",
        "data/kw_chatbot_data - 커리큘럼(VT).json",
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json",
        "data/lecture_domain.json"
    ],
    "career_counsel": [
        "data/진로상담.json",
        "data/kw_chatbot_data - 강의계획서.json",
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 커리큘럼(DS).json",
        "data/kw_chatbot_data - 커리큘럼(VT).json",
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json"],
    
    "academic_status": [
        "data/학습현황.json",
        "data/kw_chatbot_data - Student.json",
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json",
        "data/lecture_domain.json"
        ]
}

# 사용자별 컬렉션
for user_id, files in user_datasets.items():
    collection_name = f"lecture_search_{user_id}"
    try:
        client.delete_collection(name=collection_name)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=collection_name)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{collection_name}_{i}" for i in range(len(texts))]
    )
    print(f"[{user_id}] 임베딩 완료 ({len(embeddings)}건)")

# 기능별 컬렉션
for task, files in task_datasets.items():
    try:
        client.delete_collection(name=task)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=task)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{task}_{i}" for i in range(len(texts))]
    )
    print(f"[{task}] 임베딩 완료 ({len(embeddings)}건)")
    
# PDF 문서 임베딩 추가
target_collections = ["lecture_search", "career_counsel", "academic_status"]

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = PyPDFLoader("data/수강신청_자료집_전체(2025-1)v4.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs = text_splitter.split_documents(pages)
texts = [doc.page_content for doc in docs]
embeddings = embeddings_model.embed_documents(texts)

client = chromadb.PersistentClient(path="./chroma_db")

#컬렉션에 추가
for name in target_collections:
    collection = client.get_or_create_collection(name=name)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{name}_pdf_{i}" for i in range(len(texts))]
    )
    print(f" '{name}' PDF 임베딩 {len(texts)}개 추가 완료")

[kim] 임베딩 완료 (24건)
[hong] 임베딩 완료 (24건)
[lecture_search] 임베딩 완료 (617건)
[career_counsel] 임베딩 완료 (639건)
[academic_status] 임베딩 완료 (607건)
 'lecture_search' PDF 임베딩 423개 추가 완료
 'career_counsel' PDF 임베딩 423개 추가 완료
 'academic_status' PDF 임베딩 423개 추가 완료


#### 기능 구현 함수

In [211]:
import json
import re
import random
import itertools
from collections import defaultdict, Counter

# LangChain 컬렉션 이름 결정 함수
def get_collection_name(user_input: str, user_id: str) -> str:
    q = user_input.lower()
    if "진로" in q or "상담" in q:
        return "career_counsel"
    elif "이수" in q or "학점" in q or "현황" in q or "평균" in q:
        return "academic_status"
    return f"lecture_search_{user_id}"

# 입력 분류 함수
def classify_function(user_input: str) -> str:
    if "강의" in user_input or "탐색" in user_input:
        return "lecture_search"
    elif "진로" in user_input or "상담" in user_input:
        return "career_counsel"
    elif "학업" in user_input or "현황" in user_input:
        return "academic_status"
    else:
        return "lecture_search"

# 캐시: 팀플 조건에 따라 결과 재사용
TEAM_CACHE = {}

# JSON 파일에서 팀플 조건과 일치하는 강의 필터링
def filter_lectures_by_team(json_files, team_condition):
    if team_condition in TEAM_CACHE:
        return TEAM_CACHE[team_condition]

    result = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for item in data:
            team_value = item.get("lecture_team", "").strip()
            lecture_id = item.get("lecture_id")
            lecture_name = item.get("lecture_name")

            if not lecture_id or not lecture_name:
                continue

            if team_condition == "없음" and team_value in ["없음", "0"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})
            elif team_condition == "보통" and team_value in ["보통", "적은"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})
            elif team_condition == "많은" and team_value in ["많은", "많음"]:
                result.append({"lecture_id": lecture_id, "lecture_name": lecture_name})

    TEAM_CACHE[team_condition] = result
    return result

# 팀플 응답 메시지 포맷
def format_team_project_response(team_condition: str, lectures: list):
    if not lectures:
        return f"팀플이 {team_condition} 강의를 찾을 수 없습니다."

    condition_map = {
        "없음": "없는",
        "보통": "적은",
        "많은": "많은"
    }
    label = condition_map.get(team_condition, team_condition)
    header = f"팀플이 {label} 강의는 다음과 같습니다:"
    body = "\n".join(
        f"{i+1}. {lec['lecture_name']} (학정번호: {lec['lecture_id']})"
        for i, lec in enumerate(lectures)
    )
    return f"{header}\n{body}"

# JSON 내 팀플 조건 처리 핸들러
def handle_team_project_query(user_input, json_path):
    condition = normalize_team_condition(user_input)
    if not condition:
        return None

    filtered_lectures = filter_lectures_by_team([json_path], condition)
    seen = set()
    unique_lectures = []
    for lec in filtered_lectures:
        if lec["lecture_name"] not in seen:
            seen.add(lec["lecture_name"])
            unique_lectures.append(lec)

    return format_team_project_response(condition, unique_lectures)

# 학점 및 교과구분에 따른 강의 필터링
def get_lectures_by_credit_and_type(user_input: str):
    credit_match = re.search(r'(\d)\s*학점', user_input)
    if not credit_match:
        return "몇 학점짜리 수업을 원하시는지 알려주세요. 예: '2학점 교양 수업 알려줘'"
    credit = int(credit_match.group(1))

    # 수업 유형 필터링
    course_types = []
    if "전필" in user_input:
        course_types = ["전필"]
    elif "전선" in user_input:
        course_types = ["전선"]
    elif "교필" in user_input:
        course_types = ["교필"]
    elif "교선" in user_input:
        course_types = ["교선"]
    elif "전공" in user_input:
        course_types = ["전필", "전선"]
    elif "교양" in user_input:
        course_types = ["교필", "교선"]
    else:
        course_types = ["전필", "전선", "교필", "교선"]

    count_match = re.search(r'(\d+)\s*개', user_input)
    limit = int(count_match.group(1)) if count_match else None

    with open("./data/kw_chatbot_data - 수강신청자료집.json", "r", encoding="utf-8") as f:
        course_data = json.load(f)

    results = []
    seen = set()
    for item in course_data:
        course_type = item.get("lecture_course_type", "").strip()
        credit_val = item.get("lecture_hours") or item.get("lecture_credit")
        if not credit_val:
            continue
        try:
            credit_val = int(str(credit_val).strip())
        except:
            continue

        professor = item.get("lecture_professor", "").strip()
        lecture_name = item.get("lecture_name", "").strip()
        key = (lecture_name, professor)

        if course_type in course_types and credit_val == credit and key not in seen:
            seen.add(key)
            results.append(f"{lecture_name} (학정번호: {item['lecture_id']}) - {professor or '교수명 없음'}")

    if not results:
        return f"{credit}학점짜리 {', '.join(course_types)} 과목을 찾을 수 없습니다."

    if limit:
        random.shuffle(results)
        results = results[:limit]

    header = f"{credit}학점짜리 {', '.join(course_types)} 과목 목록입니다:"
    lines = [f"{i+1}. {lec}" for i, lec in enumerate(results)]
    return header + "\n" + "\n".join(lines)

# 성적 → 평점 매핑
GRADE_TO_POINT = {
    "A+": 4.5, "A0": 4.0, "B+": 3.5, "B0": 3.0,
    "C+": 2.5, "C0": 2.0, "F": 0.0
}

# 수강이력 불러오기 (ID → 파일명 매핑)
def load_student_data(user_id: str):
    filename = f"data/kw_chatbot_data - {'김브티' if user_id == 'kim' else '홍데사'}_수강이력.json"
    with open(filename, encoding='utf-8') as f:
        return json.load(f)

# 재수강 횟수 계산
def count_retake_courses(user_id: str):
    data = load_student_data(user_id)
    course_count = defaultdict(int)
    for record in data:
        if record.get("retake_status") == "R":
            course_count[record["lecture_name"]] += 1
    return course_count

# 특정 과목의 성적 조회
def get_course_grade(user_id: str, course_name: str):
    data = load_student_data(user_id)
    for record in data:
        if record["lecture_name"].replace(" ", "") == course_name.replace(" ", ""):
            return record["lecutre_grade"]
    return None

# 사용자 이름 매핑
USER_NAME = {
    "kim": "김브티",
    "hong": "홍데사"
}

# 평균 평점 계산 함수
# GPA 계산 함수 (재수강 반영 여부 및 전공 필터 포함)
def get_gpa_with_retake(user_id: str, exclude_pre_retake: bool = False, course_type: str = None):
    data = load_student_data(user_id)
    seen = {}
    credits, total = 0, 0
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        credit = record['lecture_credit']
        ctype = record['lecture_course_type']

        if course_type:
            if course_type == "전공" and not ctype.startswith(("전필", "전선")):
                continue
            elif course_type != "전공" and not ctype.startswith(course_type):
                continue

        if exclude_pre_retake and record.get('retake_or_delete_status') == 'Y':
            continue

        if record.get('retake_status') == 'R':
            seen[key] = (GRADE_TO_POINT[grade], credit)
        elif key not in seen:
            seen[key] = (GRADE_TO_POINT[grade], credit)

    for point, credit in seen.values():
        credits += credit
        total += point * credit

    if credits == 0:
        return "계산할 수 있는 학점이 없습니다."
    name = USER_NAME.get(user_id, user_id)
    return f"{name}님의 평균 평점은 {round(total / credits, 2)}입니다."

# 재수강한 과목 리스트 반환
def get_retake_course_names(user_id: str):
    data = load_student_data(user_id)
    names = [r['lecture_name'] for r in data if r.get('retake_or_delete_status') == 'Y']
    return names

# 재수강한 과목 개수 반환
def get_retake_course_count(user_id: str):
    return len(get_retake_course_names(user_id))

# 특정 성적을 받은 과목 조회 함수
def get_subjects_by_grade(user_id: str, target_grade: str):
    data = load_student_data(user_id)
    return [r['lecture_name'] for r in data if r['lecutre_grade'] == target_grade]

# 특정 과목의 최종 성적 반환 함수
def get_final_grade_for_subject(user_id: str, subject_name: str):
    data = load_student_data(user_id)
    matches = [r for r in data if r['lecture_name'] == subject_name]
    if not matches:
        return None
    return matches[-1]['lecutre_grade']

# 재수강 횟수 과목별로 반환
def get_retake_course_counts(user_id: str):
    data = load_student_data(user_id)
    names = [r['lecture_name'] for r in data if r.get('retake_status') == 'R']
    return Counter(names)

# 재수강 가능 과목 조회 함수 (정확하게 재수강 제외 기준 반영)
def get_possible_retake_gpa(user_id: str, focus: str = "all", list_only: bool = False, fixed_grade: str = None):
    data = load_student_data(user_id)
    seen = {}
    excluded = {}

    # 재수강 후 B0 이상 받은 과목 제외
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded[key] = True

    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        ctype = record['lecture_course_type']

        if record.get('retake_or_delete_status') == 'Y': continue
        if key in excluded: continue
        if focus == "major" and not ctype.startswith("전공"): continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5: continue
        if key not in seen:
            seen[key] = (GRADE_TO_POINT[grade], record['lecture_credit'], record['lecture_name'])

    if list_only:
        if not seen:
            return f"지금 재수강 가능한 {'전공 ' if focus == 'major' else ''}과목은 없습니다."
        return f"재수강 가능한 {'전공 ' if focus == 'major' else ''}과목: " + ", ".join([v[2] for v in seen.values()])

def check_graduation_required_courses(user_id: str, required_json_path: str):
    # 필수 과목 목록 추출
    with open(required_json_path, encoding='utf-8') as f:
        all_courses = json.load(f)

    required = {
        (item['lecture_name'], item['lecture_course_type'])
        for item in all_courses
        if item.get('lecture_course_type') in ["교필", "전필"]
    }

    # 사용자가 이수한 과목 목록
    data = load_student_data(user_id)
    taken = {
        (record['lecture_name'], record['lecture_course_type'])
        for record in data
        if record.get("retake_or_delete_status") != "Y"
    }

    taken_required = sorted(required & taken, key=lambda x: x[1])  # 이수한 필수
    missed_required = sorted(required - taken, key=lambda x: x[1])  # 미이수 필수

    def format_list(items):
        result = defaultdict(list)
        for name, typ in items:
            result[typ].append(name)
        return result

    taken_fmt = format_list(taken_required)
    missed_fmt = format_list(missed_required)

    response_parts = []
    if taken_fmt:
        parts = [f"{typ} 과목인 {', '.join(names)}" for typ, names in taken_fmt.items()]
        response_parts.append("졸업 필수 과목 중에서 " + "과 " .join(parts) + "를 이수하셨고")
    if missed_fmt:
        parts = [f"{typ} 과목인 {', '.join(names)}" for typ, names in missed_fmt.items()]
        response_parts.append(" ".join(parts) + "를 이수하지 않았습니다.")
    if not missed_required:
        response_parts = ["졸업 필수 과목을 모두 이수하셨습니다."]
    return " ".join(response_parts)

def get_remaining_credits(user_id: str, graduation_total: int = 133):
    data = load_student_data(user_id)
    total_credits = sum(
        record['lecture_credit']
        for record in data
        if record.get('retake_or_delete_status') != "Y"
    )
    remaining = graduation_total - total_credits
    name = USER_NAME.get(user_id, user_id)
    return f"{name}님의 현재까지 이수한 학점은 {total_credits}학점이며, 졸업을 위해 {remaining}학점이 더 필요합니다."

def calculate_gpa_from_raw_split(user_id: str) -> dict:
    data = load_student_data(user_id)
    name = USER_NAME.get(user_id, user_id)

    total_points, total_credits = 0, 0
    major_points, major_credits = 0, 0

    for record in data:
        if record.get('retake_or_delete_status') == 'Y':
            continue

        grade = record.get('lecutre_grade')
        point = GRADE_TO_POINT.get(grade, None)
        credit = record.get('lecture_credit', 0)
        ctype = record.get('lecture_course_type', '')

        if point is None or not isinstance(credit, (int, float)):
            continue

        # 전체 평점 누적
        total_points += point * credit
        total_credits += credit

        # 전공 평점 누적 (전필 또는 전선만)
        if ctype.startswith("전필") or ctype.startswith("전선"):
            major_points += point * credit
            major_credits += credit

    result = {}
    if total_credits > 0:
        result["전체"] = round(total_points / total_credits, 2)
    if major_credits > 0:
        result["전공"] = round(major_points / major_credits, 2)
    else:
        result["전공"] = None  # 또는 "전공 과목 없음"

    return result

# 성적 향상 가정 시 GPA 시뮬레이션 함수 개선
def simulate_retake_gpa(user_id: str, fixed_grade: str = None, focus: str = "all"):
    if fixed_grade == "A+":
        return "재수강 시 받을 수 있는 최고학점은 A0입니다."

    data = load_student_data(user_id)
    seen = {}
    excluded = set()

    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded.add(key)
        if record.get('retake_or_delete_status') == 'Y':
            excluded.add(key)

    replace_targets = {}
    base = {}
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        credit = record['lecture_credit']
        ctype = record['lecture_course_type']
        name = record['lecture_name']

        if key in excluded:
            continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5:
            continue
        if focus == 'major' and not ctype.startswith("전공"):
            continue
        if focus == 'liberal' and ctype.startswith("전공"):
            continue

        if key not in replace_targets:
            replace_targets[key] = (GRADE_TO_POINT[grade], credit, name)

    for record in data:
        key = record['lecture_id']
        if record.get('retake_or_delete_status') == 'Y':
            continue
        base[key] = (GRADE_TO_POINT.get(record['lecutre_grade'], 0.0), record['lecture_credit'])

    grade_options = [fixed_grade] if fixed_grade else ["A0", "B+", "B0", "C+", "C0"]
    result_lines = []

    for g in grade_options:
        modified = base.copy()
        label = []
        for key, (_, credit, name) in replace_targets.items():
            modified[key] = (GRADE_TO_POINT[g], credit)
            label.append(f"{name}: {g}")
        total_credits = sum(c for _, c in modified.values())
        total_points = sum(p * c for p, c in modified.values())
        avg = round(total_points / total_credits, 2)
        result_lines.append(f"{' / '.join(label)} → 예상 평점: {avg}")

    return "\n".join(result_lines)

def get_retake_reason_summary(user_id: str) -> str:
    data = load_student_data(user_id)
    results = []

    for record in data:
        name = record.get('lecture_name')
        grade = record.get('lecutre_grade')
        course_type = record.get('lecture_course_type', '')
        is_retake = record.get('retake_status') == 'R'
        is_dropped = record.get('retake_or_delete_status') == 'Y'

        if is_dropped:
            continue
        if not course_type.startswith("전필"):
            continue
        if grade not in ["C+", "C0", "F"]:
            continue

        msg = f"{name}에서 {grade}를 받으셨습니다"
        if is_retake:
            msg += ", 재수강하셨습니다"
        results.append(msg)

    if not results:
        return "성적이 낮은 전공 필수 과목은 없습니다."

    return "전공 필수 과목 중 성적이 낮은 과목은 " + " / ".join(results) + "."

    # GPA simulation part
    original_data = load_student_data(user_id)
    base = {}
    for r in original_data:
        key = r['lecture_id']
        if r.get('retake_or_delete_status') == 'Y': continue
        base[key] = (GRADE_TO_POINT.get(r['lecutre_grade'], 0.0), r['lecture_credit'])

    replace_targets = list(seen.items())
    grade_options = [fixed_grade] if fixed_grade else list(GRADE_TO_POINT.keys())
    responses = []

    for combo in itertools.product(grade_options, repeat=len(replace_targets)):
        modified = base.copy()
        label = []
        for (key, (_, credit, name)), g in zip(replace_targets, combo):
            modified[key] = (GRADE_TO_POINT[g], credit)
            label.append(f"{name}: {g}")
        total_credits = sum(c for _, c in modified.values())
        total_points = sum(p * c for p, c in modified.values())
        avg = round(total_points / total_credits, 2)
        responses.append(f"{' / '.join(label)} → 예상 평점: {avg}")

    return "\n".join(responses)

#### ChromaDB 검색

In [223]:
import json
import re
import random
import itertools
from collections import defaultdict, Counter
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from functools import lru_cache

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)

# 재수강 가능 과목 필터링 함수 (전공/교양 구분 포함)
def get_filtered_retake_courses(user_id: str, focus: str = "all") -> list:
    data = load_student_data(user_id)
    seen = {}
    excluded = set()

    # 재수강했고 성적이 B0 이상이면 제외
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        if record.get('retake_status') == 'R' and GRADE_TO_POINT.get(grade, 0) >= 3.0:
            excluded.add(key)
        if record.get('retake_or_delete_status') == 'Y':
            excluded.add(key)

     # 재수강 조건에 부합하는 과목 수집
    for record in data:
        key = record['lecture_id']
        grade = record['lecutre_grade']
        ctype = record['lecture_course_type']
        name = record['lecture_name']

        if key in excluded:
            continue
        if GRADE_TO_POINT.get(grade, 5) > 2.5:
            continue
        if focus == 'major' and not ctype.startswith("전공"):
            continue
        if focus == 'liberal' and ctype.startswith("전공"):
            continue
        if key not in seen:
            seen[key] = name

    return list(seen.values())

# 팀플 조건별 응답 포맷 구성
def format_team_project_response(team_condition: str, lectures: list):
    if not lectures:
        return f"팀플이 {team_condition} 강의를 찾을 수 없습니다."

    condition_map = {
    "없음": "없는",
    "보통": "적은",
    "많은": "많은"
    }
    label = condition_map.get(team_condition, team_condition)
    header = f"팀플이 {label} 강의는 다음과 같습니다:"
    body = "\n".join(
        f"{i+1}. {lec['lecture_name']} (학정번호: {lec['lecture_id']})"
        for i, lec in enumerate(lectures)
    )
    return f"{header}\n{body}"

# 조별과제/팀플 조건 파싱 및 정규화
def normalize_team_condition(user_input: str) -> str | None:
    condition_map = {
        "없음": ["팀플 없는", "팀플이 없는", "조별과제 없는", "조별과제가 없는"],
        "보통": ["팀플 보통", "팀플이 보통", "조별과제 보통", "조별과제가 보통", "팀플 적은", "팀플이 적은", "조별과제 적은", "조별과제가 적은"],
        "많은": ["팀플 많은", "팀플이 많은", "조별과제 많은", "조별과제가 많은", "팀플 많음", "조별과제 많음"]
    }
    for condition, variants in condition_map.items():
        if any(v in user_input for v in variants):
            return condition
    return None
    
# 자연어 질의 응답 처리 함수
def answer_query(user_input: str, user_id: str = "kim"):
    lowered = user_input.lower()
    normalized = lowered.replace(" ", "")   # 공백 제거하여 패턴 매칭에 유리하게 처리

    # 졸업논문/프로젝트 관련 질의 선처리
    graduation_keywords = ["졸업논문", "졸업프로젝트", "졸프", "졸작", "졸업과제"]
    
    # 시기 관련 질의는 가장 먼저 처리
    if all(k in lowered for k in ["졸업", "논문", "프로젝트"]) and any(w in lowered for w in ["언제", "언제부터", "시기", "몇학년", "준비", "시작"]):
        return ("빠르면 3학년부터 준비하는 경우도 있지만, 대부분 3학년 2학기부터 팀을 구한 후 4학년 때 프로젝트를 진행합니다. 논문의 경우도 대부분 4학년 때 준비합니다.")
    
    # 필수 여부
    if all(k in lowered for k in ["졸업", "논문", "프로젝트"]) and any(w in lowered for w in ["필수", "의무", "꼭", "반드시", "필요", "해야", "해야해"]):
        return "졸업하기 위해서는 졸업 논문 혹은 졸업 프로젝트 둘 중 하나는 필수입니다."
    
    # 기타 졸프 관련 질의
    if any(k in lowered for k in graduation_keywords):
        return "졸업 논문이나 졸업 프로젝트와 관련된 사항은 전공 학과나 학교의 졸업 요건에 따라 다르며, 학과 사무실이나 지도 교수님을 통해 반드시 확인하셔야 합니다."

    # 특정 학점 + 과목 타입 필터
    if re.search(r'\d\s*학점', user_input):
        return get_lectures_by_credit_and_type(user_input)

    # 팀플 조건 분석 및 응답
    team_condition = normalize_team_condition(user_input)
    match = re.search(r'(\d+)\s*개', user_input)
    count = int(match.group(1)) if match else None

    if team_condition:
        filtered_lectures = filter_lectures_by_team(["./data/kw_chatbot_data - 강의 평점.json"], team_condition)
        seen = set()
        unique_lectures = []
        for lec in filtered_lectures:
            if lec["lecture_name"] not in seen:
                seen.add(lec["lecture_name"])
                unique_lectures.append(lec)
        if count:
            unique_lectures = unique_lectures[:count]
        return format_team_project_response(team_condition, unique_lectures)

    # 성적 이하 필터
    if "이하" in normalized:
        for grade in GRADE_TO_POINT:
            if grade.lower() in normalized:
                threshold = GRADE_TO_POINT[grade]
                target_grades = [g for g, p in GRADE_TO_POINT.items() if p <= threshold]
                subjects = [r['lecture_name'] for r in load_student_data(user_id) if r['lecutre_grade'] in target_grades]
                return f"{', '.join(subjects)} 과목이 {grade} 이하입니다." if subjects else f"{grade} 이하인 과목이 없습니다."

    # 전체 성적 평균 요청
    if any(k in normalized for k in ["전체성적평균", "전체학점평균"]):
        gpas = calculate_gpa_from_raw_split(user_id)
        if "전체" in gpas:
            return f"{USER_NAME.get(user_id, user_id)}님의 전체 성적 평균은 {gpas['전체']}입니다."
        return "전체 성적 평균은 제공된 정보에 포함되어 있지 않습니다."

    # 전공 성적 평균 요청
    if "전공" in normalized and "평균" in normalized:
        gpas = calculate_gpa_from_raw_split(user_id)
        gpa = gpas.get("전공", None)
        name = USER_NAME.get(user_id, user_id)
        if gpa is not None:
            return f"{name}님의 전공 성적 평균은 {gpa}입니다."
        return f"{name}님의 전공 성적 평균은 제공된 정보에 포함되어 있지 않습니다."

    # 재수강 횟수 조회
    if "재수강몇번했어" in normalized or "나몇번재수강했어" in normalized:
        retake_counts = count_retake_courses(user_id)
        if not retake_counts:
            return "재수강한 과목이 없습니다."
        parts = [f"{name} 과목을 {count}번" for name, count in retake_counts.items()]
        return f"{', '.join(parts)} 재수강하셨습니다."

    # 단일 과목 성적 조회
    if "성적이어때" in normalized or "학점이어때" in normalized:
        course_name = user_input.split()[0].replace(" ", "")
        grade = get_course_grade(user_id, course_name)
        return f"{course_name}의 성적은 {grade}입니다." if grade else f"{course_name}에 대한 성적 정보를 찾을 수 없습니다."

    # 특정 성적 받은 과목 나열
    for grade in GRADE_TO_POINT:
        if grade.lower() in normalized and ("받은과목" in normalized or "받은수업" in normalized):
            subjects = [r['lecture_name'] for r in load_student_data(user_id) if r['lecutre_grade'] == grade]
            return f"{', '.join(subjects)} 과목에서 {grade}를 받으셨습니다." if subjects else f"{grade}를 받은 과목이 없습니다."

    # 재수강 목록 조회
    if "재수강한거뭐있어" in normalized or "재수강한과목" in normalized:
        data = load_student_data(user_id)
        names = [r['lecture_name'] for r in data if r.get('retake_status') == 'R']
        return ", ".join(f"{name}을 재수강했습니다." for name in names) if names else "재수강한 과목이 없습니다."

    # 재수강 반영한 평점 요청
    if any(k in normalized for k in ["재수강하고전체학점", "재수강반영된학점", "재수강적용학점", "재수강하고학점"]):
        return get_gpa_with_retake(user_id, exclude_pre_retake=True)

    # 재수강 가능한 전공 과목 요청
    if "재수강가능한전공과목" in normalized and any(k in normalized for k in ["뭐가있어", "뭐있어", "뭐야"]):
        filtered = get_filtered_retake_courses(user_id, focus="major")
        return f"지금 재수강 가능한 전공 과목은 {', '.join(filtered)}입니다." if filtered else "지금 재수강 가능한 전공 과목은 없습니다."

    # 전체 재수강 가능한 과목
    if "재수강가능한과목" in normalized and any(k in normalized for k in ["뭐가있어", "뭐있어", "뭐야"]):
        majors = get_filtered_retake_courses(user_id, focus="major")
        liberals = get_filtered_retake_courses(user_id, focus="liberal")
        total = majors + liberals
        return f"지금 재수강 가능한 과목은 {', '.join(total)}입니다." if total else "지금 재수강 가능한 과목은 없습니다."

    if "재수강가능한횟수" in normalized or ("재수강" in normalized and "횟수" in normalized):
        data = load_student_data(user_id)
        retake_count = sum(1 for r in data if r.get("retake_status") == "R" and r.get("lecutre_grade") != "F")
        remaining = 8 - retake_count
        name = USER_NAME.get(user_id, user_id)
        return f"{name}님은 총 8번까지 재수강이 가능하며, 현재까지 {retake_count}번 재수강하셨습니다. 따라서 {remaining}번 더 재수강할 수 있습니다."

    # 성적 향상 가정 시 GPA 시뮬레이션
    if ("재수강가능한과목" in normalized or "재수강가능한전공과목" in normalized) and "성적올리면" in normalized:
        focus = "major" if "전공" in normalized else "all"
        fixed_grade = None
        for grade in GRADE_TO_POINT:
            if f"{grade.lower()}로" in normalized:
                fixed_grade = grade
                break
        return simulate_retake_gpa(user_id, fixed_grade=fixed_grade, focus=focus)

    if "조기졸업" in normalized and "가능" in normalized:
        data = load_student_data(user_id)
        credits = sum(r['lecture_credit'] for r in data if r.get('retake_or_delete_status') != "Y")
        grades = [GRADE_TO_POINT.get(r['lecutre_grade'], 0.0) for r in data if r.get('retake_or_delete_status') != "Y"]
        avg = round(sum(grades) / len(grades), 2) if grades else 0
    
        # C+ 이하 여부 체크
        bad_grades = {"C+", "C0", "F"}
        has_c_or_lower = any(
            r.get('lecutre_grade') in bad_grades
            for r in data if r.get('retake_or_delete_status') != "Y"
        )
    
        if credits >= 114 and avg >= 4.3 and not has_c_or_lower:
            return "현재 조기 졸업 조건을 충족하셨습니다. 신청이 가능합니다."
        else:
            message = (
                "조기 졸업 가능 조건은 6학기 말까지 114학점 이상 취득(계절수업 제외), "
                "7개 학기 이상 이수자로 학칙시행세칙 제16조 평점평균의 산출에 따른 총 평점평균이 4.3 이상인 자(학적부성적 기준), "
                "조기졸업 신청 당해 학기까지 취득한 전과목 성적이 B0 이상인 자(학적부성적 기준)입니다."
            )
            if has_c_or_lower:
                message += " 현재 C+ 이하의 성적이 존재하므로 재수강을 통해 성적을 개선하고 평점 평균을 4.3 이상으로 만든다면 조기 졸업이 가능합니다."
            return message

    if "장학금" in normalized:
        if any(k in normalized for k in ["화도", "동해"]) and "최소" in normalized and "학점" in normalized:
            return "화도·동해 장학금은 직전 학기 12학점 이상, 평점 2.0 이상이어야 신청 가능합니다."
        if "성적" in normalized:
            return "성적 장학금은 평량평균이 3.0 이상이며, 17학점 이상 취득(4학년은 12학점), 그 중 60% 이상이 전공 학점일 경우 자동 신청됩니다."

    if "졸업필수과목" in normalized:
        return check_graduation_required_courses(user_id, "data/kw_chatbot_data - 수강신청자료집.json")

    if "졸업하려면몇학점남았어" in normalized:
        return get_remaining_credits(user_id)
        
    ####################
    # 단순 고정 응답 처리
    
    if "졸업유예" in normalized:
        return "아니요. 우리 학교에서는 졸업 유예 신청이 불가능합니다."

    if "재수강" in normalized and "표시" in normalized:
        return "재수강한 과목은 성적표에 ‘R’(Retake)로 표기되며, 재수강한 과목의 성적만 성적 및 학점에 반영됩니다."

    if "f받고재수강" in normalized or ("f" in normalized and "재수강" in normalized):
        return "F 받은 과목은 재수강 시 '재수강' 표시가 뜨지 않지만, 재수강 이력은 남고 성적표에는 'R'로 표기됩니다."

    if "동일과목" in normalized and "몇번" in normalized:
        return "C+ 이하인 동일과목은 최대 2회까지 재수강 가능하며, 전체 재학 중 재수강 가능한 교과목 수는 최대 8과목입니다. F 학점은 제외됩니다."
        
    if "휴학" in normalized and "몇번" in normalized:
        return "일반휴학을 최대 6학기(3년) 할 수 있으며, 한 번 신청 시 최대 2학기 가능합니다. 군휴학 및 창업휴학은 일반휴학 기간에 포함되지 않습니다. 특별한 사유가 있을 경우 추가 휴학이 가능할 수도 있습니다. (단, 일반휴학기간은 한 번에 두 학기를 초과하지 못하며, 통산 3년(6학기)(연속해서는 2년)을 초과하지 못합니다.)"

    if "같은과목" in normalized and "재수강" in normalized and ("교수" in normalized or "사라지" in normalized or "폐지" in normalized):
        return "학정번호 기준으로 동일한 학정번호가 있는 강의를 수강하시면 됩니다. 다만, 동일한 학정번호의 강의가 3년 이상 미개설 할 경우, C이하의 과목은 학점삭제 가능합니다. 자세한 내용은 학사에 문의하세요."
    ####################    
    
    #재수강 vs 새로운 전공과목 선택 질문
    if any(x in normalized for x in ["재수강", "다시듣는", "다시 들어야", "낮은성적"]) and any(y in normalized for y in ["전공", "필수", "과목", "수업"]):
        data = load_student_data(user_id)
        low_grade_threshold = 2.5
        low_major_required = []

        for record in data:
            grade_str = record.get("lecutre_grade")
            grade = GRADE_TO_POINT.get(grade_str, 5)
            ctype = record["lecture_course_type"]
            name = record["lecture_name"]

            if not ctype.startswith("전"):
                continue
            if grade > low_grade_threshold:
                continue
            if record.get("retake_or_delete_status") == "Y":
                continue

            low_major_required.append((grade_str, name))

        name = USER_NAME.get(user_id, user_id)
        response = f"재수강 여부는 개인의 학업 목표와 상황에 따라 다르지만, 성적이 낮은 전공 과목을 재수강하는 것은 전공에 대한 이해도를 높이고 평점 평균을 개선하는 데 도움이 될 수 있습니다. 반면, 다른 전공 과목을 듣는다면 다양한 지식을 쌓을 수 있는 기회가 될 수 있습니다."

        if not low_major_required:
            response += f" 현재 성적을 고려할 때, 성적이 낮은 전공 과목은 확인되지 않았습니다."
        else:
            parts = [f"{grade}를 받은 {name} 과목" for grade, name in low_major_required]
            response += " 현재 성적을 고려할 때, 재수강이 가능한 전공 과목으로는 " + " / ".join(parts) + "이 있습니다."

        return response
   
    if "이수한과목" in normalized or "이수한수업" in normalized or "과목리스트" in normalized or "수업리스트" in normalized or "들은과목" in normalized or "지금까지이수한" in normalized:
        data = load_student_data(user_id)
        name = USER_NAME.get(user_id, user_id)
        grouped = defaultdict(list)
    
        for r in data:
            if r.get("retake_or_delete_status") == "Y":
                continue
            ctype = r["lecture_course_type"]
            grade = r["lecutre_grade"]
            lecture = r["lecture_name"]
            retake_mark = ", R" if r.get("retake_status") == "R" else ""
            grouped[ctype].append(f"{lecture}({grade}{retake_mark})")
    
        order = ["전필", "전선", "교필", "교선"]
        lines = [f"{ctype}: {' / '.join(grouped[ctype])}" for ctype in order if ctype in grouped]
    
        if not lines:
            return f"{name}님께서 이수하신 과목이 확인되지 않았습니다."
        return f"{name}님께서 현재까지 이수하신 과목 목록:\n" + "\n".join(lines)

    collection_name = get_collection_name(user_input, user_id)
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=collection_name,
        embedding_function=embeddings_model
    )
    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        다음은 사용자의 수강 이력 또는 학업/진로 관련 정보입니다.
        질문에 답할 때 반드시 정확한 정보를 바탕으로 하세요.

        성적은 A+, A0, B+, B0, C+, C0, F 등의 형식이며, 
        ❗ 절대 유사한 등급을 포함하거나 추론하지 마세요.
        예: "A+"를 요청했을 경우, 반드시 성적이 "A+"인 과목만 포함하세요. "A0"는 포함하면 안 됩니다.

        📌 답변 형식 지침:
        - 질문에 해당하는 내용이 **여러 개인 경우 빠짐없이 모두 나열**하세요.
        - 질문 유형에 따라 말끝을 다르게 쓰세요.
          예:
            - "내가 A+ 받은 과목 알려줘" → "OOO, OOO 과목에서 A+를 받으셨습니다."
            - "자료구조 성적이 뭐야?" → "자료구조의 성적은 B+입니다."
            - "객체지향프로그래밍이 전선이지?" → "네, 객체지향프로그래밍은 전필입니다."
            - "전체 평균이 어떻게 돼?" → "전체 성적 평균은 3.58입니다."
        - **절대로 "답변:"으로 시작하지 마세요. 문장은 바로 시작하세요.**
        - 정중하고 간결하게, 핵심만 자연스럽게 응답하세요.
        - 축하 인사, 추가 질문 권유 등은 하지 마세요.

        문서:
        {context}

        질문:
        {question}

        위 지침을 따라 **답변 접두어 없이** 자연스럽게 말문을 여세요.
        """
    )
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )
    return qa.invoke({"query": user_input})["result"]

#### 학점 계산

In [213]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="hong")

'홍데사님의 전체 성적 평균은 3.7입니다.'

In [37]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="kim")

'김브티님의 전체 성적 평균은 3.72입니다.'

In [52]:
answer_query("전공 평균이 어떻게 돼?", user_id="hong")

'홍데사님의 전공 성적 평균은 3.45입니다.'

In [49]:
answer_query("전공 성적 평균이 어떻게 돼?", user_id="kim")

'김브티님의 전공 성적 평균은 3.46입니다.'

In [210]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.85'

In [209]:
answer_query("지금 재수강가능한 과목들을 B0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: B0 / 정보디자인프로그래밍: B0 → 예상 평점: 3.76'

In [214]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="hong")

'프로그래밍기초: A0 / 객체지향프로그래밍: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.89'

#### 재수강

In [18]:
answer_query("재수강 몇 번 했어?", user_id="hong")

'창의설계입문 과목을 1번 재수강하셨습니다.'

In [19]:
answer_query("나 몇 번 재수강했어?", user_id="kim")

'프로그래밍기초 과목을 1번 재수강하셨습니다.'

In [95]:
answer_query("재수강한 과목이 뭐야?", user_id="kim")

'프로그래밍기초을 재수강했습니다.'

In [21]:
answer_query("재수강한 거 뭐있어?", user_id="hong")

'창의설계입문을 재수강했습니다.'

In [77]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 과목은 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍입니다.'

In [26]:
answer_query("지금 재수강가능한 전공과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 전공 과목은 없습니다.'

In [27]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="kim")

'지금 재수강 가능한 과목은 AI수학, 정보디자인프로그래밍입니다.'

In [28]:
answer_query("지금 재수강가능한 전공과목은 뭐야?", user_id="kim")

'지금 재수강 가능한 전공 과목은 없습니다.'

#### 과목 질문

In [29]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="kim")

'경제와경영, 대학수학및연습2, 인터넷활용, 생활속의생명과학 과목에서 A+를 받으셨습니다.'

In [30]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="hong")

'컴퓨팅사고, 대학수학및연습1, 그래픽디자인, 대학수학및연습2, AI수학, 화폐와금융의과거,현재그리고미래, 생활속의생명과학 과목에서 A+를 받으셨습니다.'

In [31]:
answer_query("고급C프로그래밍 학점이 어때?", user_id="kim")

'고급C프로그래밍의 성적은 B+입니다.'

In [32]:
answer_query("객체지향프로그래밍 성적이 뭐야?", user_id="kim")

'객체지향프로그래밍의 성적은 B0입니다.'

In [33]:
answer_query("C+ 이하 과목만 보고 싶어", user_id="hong")

'창의설계입문, 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍 과목이 C+ 이하입니다.'

#### 강의탐색

In [222]:
print(answer_query("팀플이 없는 강의 알려줘"))

팀플이 없는 강의는 다음과 같습니다:
1. 컴퓨터네트워크 (학정번호: I040-2-1655-01)
2. AI수학 (학정번호: I040-2-3688-01)
3. 객체지향프로그래밍 (학정번호: I040-2-7777-01)
4. 오픈소스소프트웨어 (학정번호: I040-2-8461-01)
5. 인터랙티브미디어개론 (학정번호: I040-2-9160-01)
6. 컴퓨터그래픽스 (학정번호: I040-3-3951-01)
7. 컴퓨터비전 (학정번호: I040-3-5474-01)
8. 실험설계및분석 (학정번호: I040-3-9617-01)
9. 영상AI생성모델 (학정번호: I040-4-5471-01)


In [216]:
print(answer_query("팀플 없는 강의 알려줘"))

팀플이 없는 강의는 다음과 같습니다:
1. 컴퓨터네트워크 (학정번호: I040-2-1655-01)
2. AI수학 (학정번호: I040-2-3688-01)
3. 객체지향프로그래밍 (학정번호: I040-2-7777-01)
4. 오픈소스소프트웨어 (학정번호: I040-2-8461-01)
5. 인터랙티브미디어개론 (학정번호: I040-2-9160-01)
6. 컴퓨터그래픽스 (학정번호: I040-3-3951-01)
7. 컴퓨터비전 (학정번호: I040-3-5474-01)
8. 실험설계및분석 (학정번호: I040-3-9617-01)
9. 영상AI생성모델 (학정번호: I040-4-5471-01)


In [224]:
print(answer_query("조별과제가 적은 강의 알려줘"))

팀플이 적은 강의는 다음과 같습니다:
1. 빅데이터프로그래밍 (학정번호: I040-2-4501-01)
2. 텍스트마이닝 (학정번호: I040-3-4139-01)
3. 딥러닝프로그래밍 (학정번호: I040-4-4434-01)
4. 산학협력캡스톤설계1 (학정번호: I040-4-8995-01)
5. 데이터시각화 (학정번호: I040-4-9925-01)


In [225]:
print(answer_query("조별과제 적은 강의 알려줘"))

팀플이 적은 강의는 다음과 같습니다:
1. 빅데이터프로그래밍 (학정번호: I040-2-4501-01)
2. 텍스트마이닝 (학정번호: I040-3-4139-01)
3. 딥러닝프로그래밍 (학정번호: I040-4-4434-01)
4. 산학협력캡스톤설계1 (학정번호: I040-4-8995-01)
5. 데이터시각화 (학정번호: I040-4-9925-01)


In [226]:
print(answer_query("조별과제 많은 강의 알려줘"))

팀플이 많은 강의는 다음과 같습니다:
1. IoT시스템설계및실습 (학정번호: I040-3-4503-01)
2. UX/UI디자인 (학정번호: I040-3-7737-01)
3. 기계학습 (학정번호: I040-3-9151-01)


In [221]:
print(answer_query("조별과제가 많은 강의 알려줘"))

팀플이 많은 강의는 다음과 같습니다:
1. IoT시스템설계및실습 (학정번호: I040-3-4503-01)
2. UX/UI디자인 (학정번호: I040-3-7737-01)
3. 기계학습 (학정번호: I040-3-9151-01)


In [45]:
print(answer_query("전선 중 3학점 5개만 추천해줘"))

3학점짜리 전선 과목 목록입니다:
1. 데이터시각화 (학정번호: I040-4-9925-01) - 조재희
2. 인체데이터분석및실습 (학정번호: I000-2-5715-01) - 고찬영
3. 공학설계입문 (학정번호: I000-1-2957-02) - 임재한
4. 제어알고리즘설계및실습 (학정번호: I000-2-5702-01) - 고찬영
5. 디지털헬스케어인공지능분석실무 (학정번호: I020-2-4888-01) - 교수명 없음


In [92]:
print(answer_query("전공 중 3학점 5개만 추천해줘"))

3학점짜리 전필, 전선 과목 목록입니다:
1. 딥러닝프로그래밍 (학정번호: I040-4-4434-01) - 김준석
2. 성공적인창업계획 (학정번호: 5080-4-7882-01) - 이현경
3. 컴퓨터그래픽스 (학정번호: I040-3-3951-01) - 김동준
4. 영상AI생성모델 (학정번호: I040-4-5471-01) - 김동준
5. 창의설계입문 (학정번호: I000-1-3674-01) - 신원경


In [47]:
print(answer_query("교양 중 3학점 5개만 추천해줘"))

3학점짜리 교필, 교선 과목 목록입니다:
1. 실용중국어문법 (학정번호: 0000-2-6526-01) - 곡효운
2. 평생교육론 (학정번호: 0000-1-7468-01) - 김성길
3. C프로그래밍 (학정번호: 0000-1-0019-09) - 박상준
4. 대학수학및연습1 (학정번호: 0000-1-4625-16) - 김원주
5. 한국가족의역사와현재 (학정번호: 0000-2-4833-01) - 김인호


In [93]:
print(answer_query("교양 중 2학점 5개만 추천해줘"))

2학점짜리 교필, 교선 과목 목록입니다:
1. 탁구 (학정번호: 0000-1-1679-01) - 박현애
2. 스케이팅 (학정번호: 0000-1-2238-01) - 정재은
3. 웰니스트레이닝 (학정번호: 0000-1-5684-02) - 박인혜
4. 사회봉사1 (학정번호: 0000-1-3902-01) - 임안나
5. 저작권과스마트폰의이해 (학정번호: 0000-1-8129-01) - 건국대학교 정연덕


In [219]:
print(answer_query("3학년 1학기에 들을 만한 과목 추천해줘."))

3학년 1학기에 들을 만한 과목으로는 "고급C프로그래밍", "자료구조", "컴퓨터네트워크", "모바일프로그래밍"이 있습니다. 이 과목들은 전공 필수 및 선택 과목으로, 정보융합학부에서 학습하는 데 도움이 될 것입니다.


#### 학업현황

###### 졸업

In [118]:
answer_query("졸업 필수 과목을 다 들었어?", user_id="hong")

'교필 과목인 광운인되기를 이수하지 않았습니다.'

In [117]:
answer_query("졸업하려면 몇 학점 남았어?", user_id="hong")

'홍데사님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.'

In [156]:
answer_query("졸업 논문이나 졸업 프로젝트가 필수야??", user_id="hong")

'졸업하기 위해서는 졸업 논문 혹은 졸업 프로젝트 둘 중 하나는 필수입니다.'

In [155]:
answer_query("졸업 논문 필수야??", user_id="hong")

'졸업 논문은 필수입니다.'

In [154]:
answer_query("졸업 논문이나 프로젝트는 언제부터 준비해야 할까?", user_id="hong")

'빠르면 3학년부터 준비하는 경우도 있지만, 대부분 3학년 2학기부터 팀을 구한 후 4학년 때 프로젝트를 진행합니다. 논문의 경우도 대부분 4학년 때 준비합니다.'

In [79]:
answer_query("졸업까지 최소 몇 학기 남았어?", user_id="kim")

'졸업까지 최소 4학기가 남았습니다.'

In [149]:
answer_query("우리 학교에서 졸업 유예 신청이 가능해?", user_id="hong")

'아니요. 우리 학교에서는 졸업 유예 신청이 불가능합니다.'

In [152]:
answer_query("조기 졸업이 가능한 상황이야?", user_id="hong")

'조기 졸업 가능 조건은 6학기 말까지 114학점 이상 취득(계절수업 제외), 7개 학기 이상 이수자로 학칙시행세칙 제16조 평점평균의 산출에 따른 총 평점평균이 4.3 이상인 자(학적부성적 기준), 조기졸업 신청 당해 학기까지 취득한 전과목 성적이 B0 이상인 자(학적부성적 기준)입니다. 현재 C+ 이하의 성적이 존재하므로 재수강을 통해 성적을 개선하고 평점 평균을 4.3 이상으로 만든다면 조기 졸업이 가능합니다.'

##### 재수강

In [131]:
answer_query("지금 재수강가능한 과목들을 A0로 성적올리면 전체 학점이 어떻게 돼?", user_id="kim")

'AI수학: A0 / 정보디자인프로그래밍: A0 → 예상 평점: 3.85'

In [151]:
answer_query("지금 재수강가능한 과목은 뭐야?", user_id="hong")

'지금 재수강 가능한 과목은 프로그래밍기초, 객체지향프로그래밍, 정보디자인프로그래밍입니다.'

In [176]:
answer_query("재수강 가능한 횟수가 어떻게 돼?", user_id="hong")

'홍데사님은 총 8번까지 재수강이 가능하며, 현재까지 1번 재수강하셨습니다. 따라서 7번 더 재수강할 수 있습니다.'

In [177]:
answer_query("재수강 가능한 횟수가 어떻게 돼?", user_id="kim")

'김브티님은 총 8번까지 재수강이 가능하며, 현재까지 1번 재수강하셨습니다. 따라서 7번 더 재수강할 수 있습니다.'

In [178]:
answer_query("내가 재수강한 과목이 뭔데?", user_id="kim")

'프로그래밍기초을 재수강했습니다.'

In [171]:
answer_query("같은 과목을 재수강할 때 교수님이 바뀌거나 과목이 사라지면 어떻게 해야 해?", user_id="hong")

'학정번호 기준으로 동일한 학정번호가 있는 강의를 수강하시면 됩니다. 다만, 동일한 학정번호의 강의가 3년 이상 미개설 할 경우, C이하의 과목은 학점삭제 가능합니다. 자세한 내용은 학사에 문의하세요.'

In [175]:
answer_query("재수강하면 성적표에 재수강 여부가 표시되나?", user_id="hong")

'재수강한 과목은 성적표에 ‘R’(Retake)로 표기되며, 재수강한 과목의 성적만 성적 및 학점에 반영됩니다.'

In [174]:
answer_query("F받고 재수강하면 재수강 이력이 남아?", user_id="hong")

"F 받은 과목은 재수강 시 '재수강' 표시가 뜨지 않지만, 재수강 이력은 남고 성적표에는 'R'로 표기됩니다."

In [172]:
answer_query("성적이 낮은 전공 필수 과목을 재수강하는 게 나을까, 다른 전공 과목을 듣는 게 나을까?", user_id="hong")

'재수강 여부는 개인의 학업 목표와 상황에 따라 다르지만, 성적이 낮은 전공 과목을 재수강하는 것은 전공에 대한 이해도를 높이고 평점 평균을 개선하는 데 도움이 될 수 있습니다. 반면, 다른 전공 과목을 듣는다면 다양한 지식을 쌓을 수 있는 기회가 될 수 있습니다. 현재 성적을 고려할 때, 재수강이 가능한 전공 과목으로는 C+를 받은 객체지향프로그래밍 과목 / C+를 받은 정보디자인프로그래밍 과목이 있습니다.'

In [173]:
answer_query("동일 과목은 재수강 몇 번 할 수 있어?", user_id="hong")

'C+ 이하인 동일과목은 최대 2회까지 재수강 가능하며, 전체 재학 중 재수강 가능한 교과목 수는 최대 8과목입니다. F 학점은 제외됩니다.'

##### 학업현황

In [87]:
print(answer_query("지금까지 이수한 과목 리스트를 볼 수 있을까?", user_id="hong"))

홍데사님께서 현재까지 이수하신 과목 목록:
전필: 객체지향프로그래밍(C+) / AI수학(A+) / 자료구조(B+) / 모바일프로그래밍(B+)
전선: 고급C프로그래밍(B+) / 이산수학(B0) / 그래픽디자인(A+) / 창의설계입문(A0, R) / 빅데이터프로그래밍(B0) / 데이터베이스(B+) / 정보디자인프로그래밍(C+)
교필: 프로그래밍기초(C+) / 융합적사고와글쓰기(B+) / 컴퓨팅사고(A+)
교선: 대학수학및연습1(A+) / 스케이팅(B+) / 대학수학및연습2(A+) / 아카펠라(A0) / 범죄학(B+) / 대중문화와삶(B+) / 화폐와금융의과거,현재그리고미래(A+) / 사회학의이해(A0) / 생활속의생명과학(A+)


In [179]:
print(answer_query("지금까지 이수한 과목 리스트를 볼 수 있을까?", user_id="kim"))

김브티님께서 현재까지 이수하신 과목 목록:
전필: 객체지향프로그래밍(B0) / AI수학(C+) / 자료구조(B+) / 모바일프로그래밍(A0)
전선: 창의설계입문(A0) / 고급C프로그래밍(B+) / 이산수학(A0) / 그래픽디자인(B+) / 컴퓨터네트워크(A0) / 인터랙티브미디어개론(B0) / 인터랙티브심리학(A0) / 정보디자인프로그래밍(C+)
교필: 프로그래밍기초(A0, R)
교선: 대학수학및연습1(B+) / 초급중국어1(A0) / 경제와경영(A+) / 자연과학사(B+) / 대학수학및연습2(A+) / 환경과생태(B+) / 논리적으로사고하기(B+) / 인터넷활용(A+) / 사회학의이해(A0) / 생활속의생명과학(A+)


In [78]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="hong")

'홍데사님의 전체 성적 평균은 3.7입니다.'

##### 장학금&휴학

In [186]:
answer_query("성적 장학금을 받기 위한 조건이 뭐야?", user_id="hong")

'성적 장학금은 평량평균이 3.0 이상이며, 17학점 이상 취득(4학년은 12학점), 그 중 60% 이상이 전공 학점일 경우 자동 신청됩니다.'

In [185]:
answer_query("화도 동해 장학금 최소 학점이 얼마야? ", user_id="hong")

'화도·동해 장학금은 직전 학기 12학점 이상, 평점 2.0 이상이어야 신청 가능합니다.'

In [184]:
answer_query("휴학은 최대 몇 번까지 가능해?", user_id="hong")

'일반휴학을 최대 6학기(3년) 할 수 있으며, 한 번 신청 시 최대 2학기 가능합니다. 군휴학 및 창업휴학은 일반휴학 기간에 포함되지 않습니다. 특별한 사유가 있을 경우 추가 휴학이 가능할 수도 있습니다. (단, 일반휴학기간은 한 번에 두 학기를 초과하지 못하며, 통산 3년(6학기)(연속해서는 2년)을 초과하지 못합니다.)'

#### 진로상담

In [220]:
print(answer_query("현재 DS 전공을 하고 있는데 직무를 못 정하고 있는데 어떻게 해야 할까?"))

전공과 관련된 직무를 결정하는 데 도움이 될 수 있는 몇 가지 방법이 있습니다. 우선, 본인이 수강한 과목들을 다시 살펴보며 어떤 과목에서 흥미를 느꼈는지, 또는 어떤 과목에서 좋은 성적을 받았는지를 고려해보세요. 예를 들어, 경제와경영에서 A+를 받으셨고, 대학수학및연습1과 대학수학및연습2에서도 A+를 받으셨습니다. 이러한 과목들은 데이터 분석이나 경제 관련 직무에 적합할 수 있습니다.

또한, 다양한 직무에 대한 정보를 수집하고, 인턴십이나 프로젝트에 참여하여 실무 경험을 쌓는 것도 좋습니다. 이를 통해 자신에게 맞는 직무를 찾는 데 도움이 될 것입니다. 마지막으로, 멘토나 교수님과 상담하여 조언을 받는 것도 좋은 방법입니다.


#### 유사도 검사

In [132]:
from langchain_community.vectorstores import Chroma

query = "나 졸업하려면 몇학점 남았어?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search_kim",
    embedding_function=embeddings_model
)

# 유사 문서 검색 (상위 5개)
similar_docs = db.similarity_search_with_score(query, k=3)

# 결과 출력
for doc, score in similar_docs:
    print(answer_query(query, user_id="kim"))
    print(f"\n유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

C:\Users\문선장\AppData\Local\Temp\ipykernel_5764\2873229418.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(


김브티님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.

유사도: 1.2532
학번: 2024204708
강의명: 프로그래밍기초
학정번호: I040-1-8610-01
개설 학과: 정보융합학부
이수 구분: 교필
학점: 3학점
성적: C0
재수강/삭제 여부: Y ...
김브티님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.

유사도: 1.2807
학번: 2024204708
강의명: 프로그래밍기초
학정번호: I040-1-8610-01
개설 학과: 정보융합학부
이수 구분: 교필
학점: 3학점
성적: A0
재수강 여부: R ...
김브티님의 현재까지 이수한 학점은 69학점이며, 졸업을 위해 64학점이 더 필요합니다.

유사도: 1.2870
학번: 2024204708
강의명: 대학수학및연습1
학정번호: 0000-1-4625-08
개설 학과: 전체공통
이수 구분: 교선
학점: 3학점
성적: B+ ...


In [195]:
from langchain_community.vectorstores import Chroma

query = "김현경 교수님 수업 추천해줘"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search_kim",
    embedding_function=embeddings_model
)

# 유사 문서 검색 (상위 5개)
similar_docs = db.similarity_search_with_score(query, k=3)

# 결과 출력
for doc, score in similar_docs:
    print(answer_query(query, user_id="kim"))
    print(f"\n유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

김현경 교수님의 수업으로는 "대학수학및연습2", "경제와경영", "생활속의생명과학", "인터넷활용"에서 A+를 받으셨습니다. 이 과목들을 추천드립니다.

유사도: 1.2029
학번: 2024204708
강의명: 사회학의이해
학정번호: 0000-1-3948-01
개설 학과: 전체공통
이수 구분: 교선
학점: 3학점
성적: A0 ...
김현경 교수님의 수업으로는 "대학수학및연습2", "경제와경영", "생활속의생명과학", "인터넷활용"에서 A+를 받으셨습니다. 이 과목들을 추천드립니다.

유사도: 1.2087
학번: 2024204708
강의명: 대학수학및연습2
학정번호: 0000-1-4626-11
개설 학과: 전체공통
이수 구분: 교선
학점: 3학점
성적: A+ ...
김현경 교수님의 수업으로는 "대학수학및연습2", "경제와경영", "생활속의생명과학", "인터넷활용"에서 A+를 받으셨습니다. 이 과목들을 추천드립니다.

유사도: 1.2135
학번: 2024204708
강의명: 환경과생태
학정번호: 0000-2-2994-01
개설 학과: 전체공통
이수 구분: 교선
학점: 3학점
성적: B+ ...


In [134]:
from langchain_community.vectorstores import Chroma

query = "목요일에 듣는 운동 교양 2개 추천해줘?"

# Chroma DB 연결
db = Chroma(
    persist_directory="./chroma_db",
    collection_name="lecture_search",
    embedding_function=embeddings_model
)

# 유사 문서 검색 (상위 5개)
similar_docs = db.similarity_search_with_score(query, k=3)

# 결과 출력
for doc, score in similar_docs:
    print(answer_query(query))
    print(f"\n유사도: {score:.4f}")
    print(doc.page_content[:300], "...")

목요일에 듣는 운동 교양 과목은 없습니다. 다른 과목에 대한 정보가 필요하시면 말씀해 주세요.

유사도: 1.2402
학정번호: 0000-3-3200-02
강의명: 운동과건강
강의평점: 
과제: 
팀플: 
성적평가정도: 
출결 방식: 
시험 횟수: 
시험 방식: 
전공 학점: 없음
교양 학점: 없음
총 학점: 없음
교수명: 
수업 시간: 목1,2
강의 유형: 교선
학점: 3시간
학기: 학기
강의 설명: 영역: 6 ...
목요일에 듣는 운동 교양 과목은 없습니다. 다른 과목에 대한 정보가 필요하시면 말씀해 주세요.

유사도: 1.2444
학정번호: 0000-3-3200-03
강의명: 운동과건강
강의평점: 
과제: 
팀플: 
성적평가정도: 
출결 방식: 
시험 횟수: 
시험 방식: 
전공 학점: 없음
교양 학점: 없음
총 학점: 없음
교수명: 
수업 시간: 토5,6
강의 유형: 교선
학점: 3시간
학기: 학기
강의 설명: 영역: 6 ...
목요일에 듣는 운동 교양 과목은 없습니다. 다른 과목에 대한 정보가 필요하시면 말씀해 주세요.

유사도: 1.2496
학정번호: 0000-3-3200-01
강의명: 운동과건강
강의평점: 
과제: 
팀플: 
성적평가정도: 
출결 방식: 
시험 횟수: 
시험 방식: 
전공 학점: 없음
교양 학점: 없음
총 학점: 없음
교수명: 
수업 시간: 목3,4
강의 유형: 교선
학점: 3시간
학기: 학기
강의 설명: 영역: 6 ...


In [135]:
print(answer_query("1 영역이 뭐야?"))

1 영역은 전체공통입니다.


In [136]:
print(answer_query("2영역이 뭐야?"))

2영역은 정보융합학부와 관련된 과목들로 구성되어 있습니다.


In [137]:
print(answer_query("3 영역이 뭐야?"))

3 영역은 전선, 교선, 전필로 구성되어 있습니다.
